In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

In [2]:
from data_d_dicts import links

df = pd.read_csv(links['d_learning_notes_url'])
df

NameError: name 'links' is not defined

In [86]:


list_ = ['Process','Categorization','Word','Definition']

# Download Data
definition_df = pd.read_csv(links['google_definition_csv'])
notes_df = pd.read_csv(links['google_notes_csv'])

definition_df = definition_df[list_].copy()
notes_df = notes_df[list_].copy()

# Step 1. Create a list of unique Processes from Notes.
process_list = notes_df['Process'].unique().tolist()
process_list.extend([x for x in definition_df['Process'].unique() if (x not in process_list)&(pd.notna(x))])
process_map = {x:count+0 for count,x in enumerate(process_list)}

# Step 2 Create a Unique Classification List
categorization_list = [
    'Definition','Guiding Principle','Consideration','Process Step','Procedure','Expected Outcomes','Parameter','Algorithm']
